In [2]:
%load_ext autoreload
%autoreload 2
from rdkit import Chem
from rdkit.Chem import Draw, rdChemReactions, AllChem, rdFMCS
from rdkit.Chem.Draw import IPythonConsole
from src.rxn_ctr_mcs import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/stef/miniconda3/envs/mine/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at /home/stef/miniconda3/envs/mine/lib/python3.7/site-packages/rxnmapper/models/transformers/albert_heads_8_uspto_all_1310k were not used when initializing AlbertModel: ['predictions.bias', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expec

In [3]:
# Develop rxns -> PRC MCS Index

pred_rxn_sma = 'O=C(O)CCC(CC(=O)O)C(=O)O.\
                NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(OP(=O)(O)O)C3O)C(O)C2O)c1\
                >>O=C(O)C=C(CCC(=O)O)C(=O)O.\
                NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(OP(=O)(O)O)C3O)C(O)C2O)C=CC1'

known_rxn_entry = [{'(2R)-2-methylbutanedioic acid:0': 'CC(CC(=O)O)C(=O)O',
                    'NAD+:0': 'NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)c1'},
                    {'NADH:0': 'NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)C=CC1',
                    'citraconic acid:0': 'CC(=CC(=O)O)C(=O)O'}]

known_rxn_sma = entry2smarts(known_rxn_entry)

rxn_sma1 = pred_rxn_sma
rxn_sma2 = known_rxn_sma

'''
'''
am_rxn_smarts = [atom_map(rxn_sma1), atom_map(rxn_sma2)] # Atom map

# Construct reaction objects
rxns = []
for elt in am_rxn_smarts:
    temp = AllChem.ReactionFromSmarts(elt, useSmiles=True)
    temp.Initialize()
    rxns.append(temp)

rc_atoms = [elt.GetReactingAtoms() for elt in rxns] # Get reaction center atom idxs

# Construct rxn ctr mol objs
rcs = []
for i, t_rxn in enumerate(rxns):
    temp = []
    for j, t_mol in enumerate(t_rxn.GetReactants()):
        temp.append(get_sub_mol(t_mol, rc_atoms[i][j]))
    rcs.append(temp)

# Align substrates of the 2 reactions
rc_idxs = [] # Each element: (idx for rxn 1, idx for rxn 2)
remaining = [[i for i in range(len(elt))] for elt in rcs]
while (len(remaining[0]) > 0) & (len(remaining[1]) > 0):
    idx_pair = align_substrates(rcs, remaining)

    if idx_pair is None:
        break
    else:
        rc_idxs.append(idx_pair)
        remaining[0].remove(idx_pair[0])
        remaining[1].remove(idx_pair[1])

# Re-label atom map nums in substrate pairs
# so that reaction centers of aligned subs
# take atommapnum from rxn1 arbitrarily
# (Necessary for PRC MCS)
rxns = align_atom_map_nums(rxns, rcs, rc_idxs, rc_atoms)

prc_mcs = get_prc_mcs(rxns, rcs, rc_idxs, rc_atoms) # Compute MCS seeded by reaction center
prc_mcs

[1.0, 0.8888888888888888]